# Извлечение навыков из поля `description`

## Импорт библиотек

In [51]:
import re
import pandas as pd

from pathlib import Path
from itertools import chain

In [52]:
# Хранилище обработанных данных:

PROCESSED_DIR = Path('../data/processed')
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


## Работа с данными

In [9]:
df = pd.read_csv("https://izemenkov.github.io/skillgrow/data/raw_data/vacancies_master.csv")

# Оставляем только описание
texts = df["description"].dropna().copy()

# Простейшая очистка
def clean_text(t):
    t = re.sub(r"<.*?>", " ", t)         # убираем html разметку, если сохранилась
    t = re.sub(r"[^а-яА-Яa-zA-Z0-9+#/\. ]", " ", t)  # убираем спецсимволы
    t = re.sub(r"\s+", " ", t)
    return t.lower().strip()

texts = texts.apply(clean_text)

df["clean_text"] = texts
df.head()

,id,name,area,experience,key_skills,description,clean_text
0,127202494,ML-аналитик / Data Scientist,Москва,Нет опыта,NaN,Обязанности: Моделирование клиентского поведе...,обязанности моделирование клиентского поведени...
1,127244399,Аналитик данных/Data Analyst,Нижний Новгород,Нет опыта,"Python, SQL, Базы данных, Анализ данных","Ищем специалиста, готового влиться в команду п...",ищем специалиста готового влиться в команду пр...
2,126209623,"Стажер, Data Analyst / Data Scientist",Москва,Нет опыта,"Python, pandas, Numpy, ООП, Алгоритмы и структ...",В сегодняшней бизнес-среде компаниям необходим...,в сегодняшней бизнес среде компаниям необходим...
3,127192646,Младший аналитик данных/Junior Data Analyst,Москва,От 1 года до 3 лет,"Python, SQL, Аналитическое мышление, Исследова...","Mediascope – исследовательская компания, котор...",mediascope исследовательская компания которая ...
4,126562421,Junior Data Analyst/Аналитик данных,Казань,Нет опыта,NaN,Привет! Мы команда разработчиков платформы You...,привет мы команда разработчиков платформы youn...


In [13]:
# Создадим список навыков
df["skills_list"] = df["key_skills"].fillna("").apply(lambda x: [s.strip().lower() for s in x.split(",") if s.strip()])

# объединяем все навыки в один список
all_skills = list(chain.from_iterable(df["skills_list"]))

skill_freq = (
    pd.Series(all_skills)
    .value_counts()
    .reset_index()
    .rename(columns={"index": "skill", 0: "count"})
)

skill_freq.head(20)


,skill,count
0,python,190
1,sql,140
2,pandas,44
3,pytorch,37
4,ml,36
5,big data,36
6,анализ данных,34
7,математическая статистика,31
8,numpy,24
9,hadoop,23


In [17]:
# Проверим на "шум"

# Навыки короче 2 символов
skill_freq[skill_freq.skill.str.len() < 2]

# Видим только r, оставим

,skill,count
58,r,5


In [20]:
# Навыки, встречающиеся менее 3 раз
skill_freq[skill_freq["count"] < 3]

,skill,count
90,svd,2
91,математический анализ,2
92,дашборд,2
93,datalens,2
94,rag,2
...,...,...
313,argo workflows,1
314,с++,1
315,разработка платформы,1
316,tdorp,1


Навыки, встречающиеся менее двух раз уберем. Среди них, визуально, есть и значащие, но, зачастую, они являются подразделами более частых навыков. Попробуем обойтись без них

In [28]:
# убираем навыки частотой < 2 и навык it, как незначащий
skill_vocab = skill_freq[(skill_freq["count"] >= 3) & (skill_freq["skill"] != 'it')]["skill"].tolist()
len(skill_vocab), skill_vocab[:30]


(89,
 ['python',
  'sql',
  'pandas',
  'pytorch',
  'ml',
  'big data',
  'анализ данных',
  'математическая статистика',
  'numpy',
  'hadoop',
  'power bi',
  'визуализация данных',
  'nlp',
  'data science',
  'machine learning',
  'tensorflow',
  'deep learning',
  'clickhouse',
  'git',
  'scikit-learn',
  'mlflow',
  'postgresql',
  'docker',
  'ms excel',
  'pyspark',
  'data analysis',
  'etl',
  'разработка поисковых технологий',
  'a/b тесты',
  'llm'])

In [29]:
# Извлекаем навыки с помощью полученного списка:

df["clean_text"] = df["description"].fillna("").str.lower()

def extract_from_text(text, vocab):
    results = []
    for s in vocab:
        if len(s) == 1:
            # skill состоит из 1 буквы: ищем только как отдельное слово
            # пример: r → ищем ` r ` или начало/конец строки
            pattern = r'\b' + re.escape(s) + r'\b'
            if re.search(pattern, text):
                results.append(s)

        else:
            # обычные навыки: разрешаем подстроки
            if s in text:
                results.append(s)

    return results

df["skills_from_description"] = df["clean_text"].apply(lambda t: extract_from_text(t, skill_vocab))


In [30]:
# Проверим покрытие
coverage = (df["skills_from_description"].apply(len) > 0).mean()
print(f"Доля вакансий с навыками из описания: {coverage:.1%}")

skills_flat = list(chain.from_iterable(df["skills_from_description"]))
pd.Series(skills_flat).value_counts().head(20)


Доля вакансий с навыками из описания: 99.6%


python          391
sql             318
ml              304
pandas          175
pytorch         141
numpy           134
llm             130
nlp             119
аналитика       118
git             115
spark           110
docker          100
api              88
scikit-learn     72
data science     70
clickhouse       65
power bi         65
pyspark          64
hadoop           62
etl              62
Name: count, dtype: int64

Удалось добиться хорошего покрытия вакансий навыками.
Теперь приедем все к единому полю навыков, с которым будем работать в дальнейшем

In [35]:
def unite_skills(row):
    s1 = row.get("skills_list", [])
    s2 = row.get("skills_from_description", [])
    # объединяем, чистим и сортируем
    return sorted(set(map(str.lower, s1 + s2)))

df["final_skills"] = df.apply(unite_skills, axis=1)


In [41]:
# Смотрим частоты по финальному списку навыков

final_skills_flat = list(chain.from_iterable(df["final_skills"]))
pd.Series(final_skills_flat).value_counts().head(25)


python           407
sql              333
ml               306
pandas           186
pytorch          150
numpy            143
llm              131
git              127
аналитика        119
nlp              119
spark            115
docker           105
api               88
data science      82
scikit-learn      79
hadoop            76
clickhouse        71
pyspark           71
power bi          71
анализ данных     68
big data          64
tensorflow        62
etl               62
postgresql        60
matplotlib        60
Name: count, dtype: int64

In [43]:
# Смотрим частотность навыков, исходя из опыта
exploded = df.explode("final_skills")
grp = exploded.groupby(["experience", "final_skills"]).size().reset_index(name="count")
grp.sort_values("count", ascending=False).groupby("experience").head(15)


,experience,final_skills,count
432,От 3 до 6 лет,python,218
451,От 3 до 6 лет,sql,173
402,От 3 до 6 лет,ml,173
226,От 1 года до 3 лет,python,137
237,От 1 года до 3 лет,sql,115
433,От 3 до 6 лет,pytorch,94
196,От 1 года до 3 лет,ml,91
421,От 3 до 6 лет,pandas,90
397,От 3 до 6 лет,llm,78
415,От 3 до 6 лет,numpy,73


### Выводы

* Удалось добиться извлечения навыков из вакансий с хорошим покрытием (>99%)
* Собрали ядро DS компетенций для рынка РФ
| Навык                                  | Частота                                                           | Комментарий                             |
| -------------------------------------- | ----------------------------------------------------------------- | --------------------------------------- |
| **python**                             | 407                                                               | базовый рабочий язык                    |
| **sql**                                | 333                                                               | must-have для аналитики и продакшена    |
| **ml**                                 | 306                                                               | обобщающее, но отражает роль            |
| **pandas / numpy**                     | 186 / 143                                                         | работа с табличными данными             |
| **pytorch / tensorflow**               | 150 / 62                                                          | модели в проде, особенно pytorch        |
| **nlp / llm**                          | 119 / 131                                                         | **всплеск из-за LLM тренда**, это важно |
| **spark / hadoop / pyspark**           | 115 / 76 / 71                                                     | Big Data → всё чаще must-have           |
| **docker / git / api**                 | 105 / 127 / 88                                                    | продакшен навыки                        |
| **power bi / clickhouse / postgresql** | Power BI = BI для бизнеса; ClickHouse/PostgreSQL                  | инфраструктура                       |


* Навыки против опыта показывают возможный карьерный трек специалиста:

| Опыт          | Ключевые навыки                                              | Интерпретация                                |
| ------------- | ------------------------------------------------------------ | -------------------------------------------- |
| **Нет опыта** | python, sql, pandas, power bi                                | Вход с BI / аналитики ↔ стажировки           |
| **1–3 года**  | python, sql, ml, pandas, nlp, numpy, docker                  | переход «анализ → модели → прототипирование» |
| **3–6 лет**   | python, sql, llm, pytorch, spark, docker, clickhouse, hadoop | продакшн ML + Big Data + архитектура         |
| **> 6 лет**   | python, sql, ml, big data, power bi, cv                      | роли "ведущий / архитектор / руководитель"   |


MVP Системы рекомендаций может работать следующим образом:

"Вы целитесь в позицию с опытом X. Средние навыки для этой зоны: ...
У вас есть: ...
Рекомендуем добрать: ..."


In [54]:
# Сохраним файл мастер-данных в ../data/processed
df_to_save = df[['id','name','area','experience','description','final_skills']]

In [55]:
df_to_save

,id,name,area,experience,description,final_skills
0,127202494,ML-аналитик / Data Scientist,Москва,Нет опыта,Обязанности: Моделирование клиентского поведе...,"[etl, ml, mlflow, pandas, python, scikit-learn..."
1,127244399,Аналитик данных/Data Analyst,Нижний Новгород,Нет опыта,"Ищем специалиста, готового влиться в команду п...","[dwh, python, sql, анализ данных, аналитика, б..."
2,126209623,"Стажер, Data Analyst / Data Scientist",Москва,Нет опыта,В сегодняшней бизнес-среде компаниям необходим...,"[big data, data science, git, ml, ms sql, nump..."
3,127192646,Младший аналитик данных/Junior Data Analyst,Москва,От 1 года до 3 лет,"Mediascope – исследовательская компания, котор...","[data science, dwh, jupyter notebook, python, ..."
4,126562421,Junior Data Analyst/Аналитик данных,Казань,Нет опыта,Привет! Мы команда разработчиков платформы You...,"[numpy, pandas, power bi, python, sql, tableau]"
...,...,...,...,...,...,...
473,126190799,ML-разработчик (рекомендательные системы),Санкт-Петербург,От 3 до 6 лет,Дзен — сложная рекомендательная система и блог...,"[catboost, java, lightgbm, ml, nlp, recsys, sv..."
474,126637726,Старший ML-разработчик в Яндекс Плюс и AdTech,Москва,От 3 до 6 лет,Яндекс Плюс — это единая подписка на сервисы Я...,"[java, ml, python, sql, алгоритмы и структуры ..."
475,126638664,Специалист по машинному обучению (middle),Нижний Новгород,От 3 до 6 лет,"Чем предстоит заниматься Cоздавать, внедрять ...","[git, linux, llm, ml, nlp, pyspark, python, r,..."
476,126638661,Специалист по машинному обучению (middle),Самара,От 3 до 6 лет,"Чем предстоит заниматься Cоздавать, внедрять ...","[git, linux, llm, ml, nlp, pyspark, python, r,..."


In [56]:
df_to_save.to_csv(PROCESSED_DIR / 'extracted_skills.csv', index=False)